###  <center><h1> Benchmark Regression  </h1></center>
----

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
data=pd.read_csv("Data/train_bm.csv")

In [8]:
data.shape

(8523, 12)

In [9]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
data.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

### shuffling and creating train and test

In [15]:
from sklearn.utils import shuffle
data=shuffle(data, random_state=42)
div=int(data.shape[0]/4)
train=data.iloc[:3*div+1,:]
test=data.iloc[3*div+1:]

In [16]:
train.shape, test.shape, data.shape

((1600, 12), (532, 12), (2132, 12))

In [18]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
2584,FDQ16,19.700,Low Fat,0.041731,Frozen Foods,109.2912,OUT035,2004,Small,Tier 2,Supermarket Type1,2402.2064
6078,FDE21,12.800,Low Fat,0.022991,Fruits and Vegetables,114.9492,OUT045,2002,NaN,Tier 2,Supermarket Type1,1737.7380
3520,FDD52,18.250,Regular,0.183260,Dairy,108.5570,OUT035,2004,Small,Tier 2,Supermarket Type1,1208.4270
3932,FDN25,7.895,Regular,0.061425,Breakfast,57.7588,OUT018,2009,Medium,Tier 3,Supermarket Type2,1145.1760
8408,FDP48,7.520,Regular,0.044272,Baking Goods,181.3950,OUT017,2007,NaN,Tier 2,Supermarket Type1,2563.3300


In [20]:
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
1274,FDG46,8.63,Regular,0.032882,Snack Foods,113.5518,OUT013,1987,High,Tier 3,Supermarket Type1,2732.4432
6898,DRJ51,14.10,Low Fat,0.087921,Dairy,228.5668,OUT013,1987,High,Tier 3,Supermarket Type1,1151.8340
5861,FDU51,20.20,Regular,0.096495,Meat,175.6028,OUT035,2004,Small,Tier 2,Supermarket Type1,3364.9532
5220,DRG39,14.15,Low Fat,0.042181,Dairy,52.0982,OUT046,1997,Small,Tier 1,Supermarket Type1,578.5802
543,FDG53,10.00,Low Fat,0.045819,Frozen Foods,138.7180,OUT013,1987,High,Tier 3,Supermarket Type1,1398.1800


### Sample_mean (item of Outlet Sales)

In [21]:
test['sample_mean']=train['Item_Outlet_Sales'].mean()

C:\Users\HP\AppData\Local\Temp\ipykernel_17160\1063202548.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['sample_mean']=train['Item_Outlet_Sales'].mean()


In [22]:
from sklearn.metrics import mean_absolute_error as MAE
sample_mean_error=MAE(test['Item_Outlet_Sales'], test['sample_mean'])
sample_mean_error

1273.8763477011278

### Mean Item outlet Sales with respect to Type

In [23]:
out_type=pd.pivot_table(train, values='Item_Outlet_Sales',index=['Outlet_Type'], aggfunc=np.mean)
out_type

C:\Users\HP\AppData\Local\Temp\ipykernel_17160\781283472.py:1: FutureWarning: The provided callable <function mean at 0x0000020E8F2559E0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  out_type=pd.pivot_table(train, values='Item_Outlet_Sales',index=['Outlet_Type'], aggfunc=np.mean)


,Item_Outlet_Sales
Outlet_Type,
Grocery Store,317.190889
Supermarket Type1,2281.555093
Supermarket Type2,1848.822453
Supermarket Type3,3526.098049


In [24]:
test['Out_type_mean'] = 0
for i in train['Outlet_Type'].unique():
  test['Out_type_mean'][test['Outlet_Type'] == str(i)] = train['Item_Outlet_Sales'][train['Outlet_Type'] == str(i)].mean()

C:\Users\HP\AppData\Local\Temp\ipykernel_17160\3265239723.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Out_type_mean'] = 0
C:\Users\HP\AppData\Local\Temp\ipykernel_17160\3265239723.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = valu

In [25]:
out_type_error = MAE(test['Item_Outlet_Sales'] , test['Out_type_mean'] )
out_type_error

1065.062604767975

In [26]:
out_year = pd.pivot_table(train, values='Item_Outlet_Sales', index = ['Outlet_Establishment_Year'], aggfunc=np.mean)
out_year

C:\Users\HP\AppData\Local\Temp\ipykernel_17160\2250129155.py:1: FutureWarning: The provided callable <function mean at 0x0000020E8F2559E0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  out_year = pd.pivot_table(train, values='Item_Outlet_Sales', index = ['Outlet_Establishment_Year'], aggfunc=np.mean)


,Item_Outlet_Sales
Outlet_Establishment_Year,
1985,2410.214430
1987,2402.062648
1997,2052.246210
1998,302.693074
1999,2181.901013
2002,2181.664311
2004,2533.671976
2007,2329.697011
2009,1848.822453


In [27]:
test['Out_year_mean'] = 0

# For every unique entry in Outlet_Identifier
for i in train['Outlet_Establishment_Year'].unique():
  # Assign the mean value corresponding to unique entry
  test['Out_year_mean'][test['Outlet_Establishment_Year'] == i] = train['Item_Outlet_Sales'][train['Outlet_Establishment_Year'] == i].mean()

C:\Users\HP\AppData\Local\Temp\ipykernel_17160\1779449868.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Out_year_mean'] = 0
C:\Users\HP\AppData\Local\Temp\ipykernel_17160\1779449868.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = valu

In [28]:
out_year_error = MAE(test['Item_Outlet_Sales'] , test['Out_year_mean'] )
out_year_error

1196.7778532181135

In [29]:
out_loc = pd.pivot_table(train, values = 'Item_Outlet_Sales', index = ['Outlet_Location_Type'], aggfunc = np.mean)
out_loc

C:\Users\HP\AppData\Local\Temp\ipykernel_17160\3691349422.py:1: FutureWarning: The provided callable <function mean at 0x0000020E8F2559E0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  out_loc = pd.pivot_table(train, values = 'Item_Outlet_Sales', index = ['Outlet_Location_Type'], aggfunc = np.mean)


,Item_Outlet_Sales
Outlet_Location_Type,
Tier 1,1714.629253
Tier 2,2355.094017
Tier 3,2217.442828


In [30]:
test['out_loc_mean'] = 0
for i in train['Outlet_Location_Type'].unique():
  test['out_loc_mean'][test['Outlet_Location_Type'] == str(i)] = train['Item_Outlet_Sales'][train['Outlet_Location_Type'] == str(i)].mean()

C:\Users\HP\AppData\Local\Temp\ipykernel_17160\3255889555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['out_loc_mean'] = 0
C:\Users\HP\AppData\Local\Temp\ipykernel_17160\3255889555.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = value

In [31]:
out_loc_error = MAE(test['Item_Outlet_Sales'] , test['out_loc_mean'] )
out_loc_error

1290.7643490613918

In [32]:
combo = pd.pivot_table(train, values = 'Item_Outlet_Sales', index = ['Outlet_Location_Type','Outlet_Establishment_Year'], aggfunc = np.mean)
combo

C:\Users\HP\AppData\Local\Temp\ipykernel_17160\3745306419.py:1: FutureWarning: The provided callable <function mean at 0x0000020E8F2559E0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  combo = pd.pivot_table(train, values = 'Item_Outlet_Sales', index = ['Outlet_Location_Type','Outlet_Establishment_Year'], aggfunc = np.mean)


Item_Outlet_Sales
Outlet_Location_Type Outlet_Establishment_Year                   
Tier 1               1985                              333.124131
                     1997                             2052.246210
                     1999                             2181.901013
Tier 2               2002                             2181.664311
                     2004                             2533.671976
                     2007                             2329.697011
Tier 3               1985                             3526.098049
                     1987                             2402.062648
                     1998                              302.693074
                     2009                             1848.822453

In [33]:
test['Super_mean'] = 0
s2 = 'Outlet_Location_Type'
s1 = 'Outlet_Establishment_Year'
for i in test[s1].unique():
  for j in test[s2].unique():
    test['Super_mean'][(test[s1] == i) & (test[s2]==str(j))] = train['Item_Outlet_Sales'][(train[s1] == i) & (train[s2]==str(j))].mean()

C:\Users\HP\AppData\Local\Temp\ipykernel_17160\4032453573.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Super_mean'] = 0
C:\Users\HP\AppData\Local\Temp\ipykernel_17160\4032453573.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values`

In [34]:
super_mean_error = MAE(test['Item_Outlet_Sales'] , test['Super_mean'] )
super_mean_error

1070.1356583075278